# Data cleaning

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import numpy as np 
import pandas as pd
import re
#import nltk
#nltk.download('stopwords')
#from nltk.corpus import stopwords
#stop_words = set(stopwords.words('english'))
#from nltk.stem import PorterStemmer 
#from nltk.tokenize import word_tokenize

#from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.stem import WordNetLemmatizer
#from nltk.stem.porter import *
#wordnet_lemmatizer = WordNetLemmatizer()


data = pd.read_csv("sample_data/data.csv")
data["Label"] = data["Label"].replace({"Non-hate": 0, "Hate":1})
data.head(5)

,Unnamed: 0,videoID,Comment,publishedAt,Channel,Label
0,0,qFAju1K_tRg,So surreal. Itâs like looking at an older ve...,2020-03-12T16:26:20Z,Breitbart,0
1,1,eI6_hJrjt6U,If you agree with Breitbart for playing this y...,2020-09-09T03:26:42Z,Breitbart,0
2,2,Op604Lh8UOI,"Why is lying cheating, stealing CIA criminal P...",2020-03-21T17:53:26Z,Breitbart,0
3,3,qik2wyLe2DA,I have always thought the dems and media's bul...,2020-04-14T12:08:07Z,Breitbart,0
4,4,aWMn_X2cED4,But but I thought they were against daily brie...,2020-05-12T21:37:47Z,Breitbart,0


In [ ]:

def process_comment(text):
    #Remove www.* or https?://*
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))\s+','',text)
    text = re.sub('\s+((www\.[^\s]+)|(https?://[^\s]+))','',text)
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    #Remove @username
    #text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = re.sub('@[^\s]+','',text)
    text = re.sub('#[^\s]+','',text)
     # specific
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    # general
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    #Remove additional white spaces
    text = re.sub('[\s]+', ' ', text)
    #Remove emojis
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    text = re.sub(r"\d+", "", text)
    text = text.lower().strip()
    return text

data["Cleaned_Comment"] = data["Comment"].apply(lambda x: process_comment(x))
data.head(5)


,Unnamed: 0,videoID,Comment,publishedAt,Channel,Label,Cleaned_Comment
0,0,qFAju1K_tRg,So surreal. Itâs like looking at an older ve...,2020-03-12T16:26:20Z,Breitbart,0,so surreal its like looking at an older versio...
1,1,eI6_hJrjt6U,If you agree with Breitbart for playing this y...,2020-09-09T03:26:42Z,Breitbart,0,if you agree with breitbart for playing this y...
2,2,Op604Lh8UOI,"Why is lying cheating, stealing CIA criminal P...",2020-03-21T17:53:26Z,Breitbart,0,why is lying cheating stealing cia criminal po...
3,3,qik2wyLe2DA,I have always thought the dems and media's bul...,2020-04-14T12:08:07Z,Breitbart,0,i have always thought the dems and media is bu...
4,4,aWMn_X2cED4,But but I thought they were against daily brie...,2020-05-12T21:37:47Z,Breitbart,0,but but i thought they were against daily brie...


In [ ]:
data.to_csv("sample_data/data-4.csv")

In [ ]:
dataset = data[["videoID", "Cleaned_Comment", "publishedAt", "Channel", "Label"]]
dataset = dataset.rename(columns={"Cleaned_Comment":"Comment"})
dataset = dataset[dataset["Comment"]!=""]
dataset = dataset.reset_index()

In [ ]:
dataset["Comment"][3045]

'ccp is far advance with nanotechnology  recently they presented their nano penises  mr ted seems to like it  he calls it revolutionary and true game changer  he finished with quote  now i dont need us funding  with the xis micropenis  i can never be unsatisfied  yes ccp has done great blowjob in keeping wuhan virus local affairs'

In [ ]:
print(data.shape)
print(dataset.shape)

(3759, 7)
(3733, 6)


In [ ]:
dataset.head(5)

,index,videoID,Comment,publishedAt,Channel,Label
0,0,qFAju1K_tRg,so surreal its like looking at an older versio...,2020-03-12T16:26:20Z,Breitbart,0
1,1,eI6_hJrjt6U,if you agree with breitbart for playing this y...,2020-09-09T03:26:42Z,Breitbart,0
2,2,Op604Lh8UOI,why is lying cheating stealing cia criminal po...,2020-03-21T17:53:26Z,Breitbart,0
3,3,qik2wyLe2DA,i have always thought the dems and media is bu...,2020-04-14T12:08:07Z,Breitbart,0
4,4,aWMn_X2cED4,but but i thought they were against daily brie...,2020-05-12T21:37:47Z,Breitbart,0


# Bert Embedding

In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import torch
import transformers as ppb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# Tokenization
#tokenized = dataset["Comment"].apply((lambda x: tokenizer.encode(x, max_length = 100,truncation=True, add_special_tokens=True)))
tokenized = dataset["Comment"].apply((lambda x: tokenizer.encode(x, max_length = 20, truncation=True, add_special_tokens=True)))

In [ ]:
# Padding

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
np.array(padded).shape

(3733, 20)

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(3733, 20)

In [ ]:
padded

array([[  101,  2061, 16524, ...,  2214, 18081,   102],
       [  101,  2065,  2017, ..., 16784,  2039,   102],
       [  101,  2339,  2003, ...,  1998,  2054,   102],
       ...,
       [  101,  2023,  8814, ...,  5974,  6176,   102],
       [  101,  2339,  2515, ...,  2137,  2024,   102],
       [  101,  1999,  2613, ...,  8814,  4819,   102]])

In [ ]:
tokenized.values

array([list([101, 2061, 16524, 2049, 2066, 2559, 2012, 2019, 3080, 2544, 1997, 18520, 2054, 1037, 3972, 14499, 2389, 2214, 18081, 102]),
       list([101, 2065, 2017, 5993, 2007, 7987, 20175, 8237, 2102, 2005, 2652, 2023, 2017, 2323, 2507, 2009, 1037, 16784, 2039, 102]),
       list([101, 2339, 2003, 4688, 16789, 11065, 9915, 4735, 13433, 8737, 8780, 2085, 4760, 2039, 2000, 2122, 9281, 1998, 2054, 102]),
       ...,
       list([101, 2023, 8814, 4819, 7865, 2003, 2886, 2668, 2828, 1037, 2111, 3531, 2668, 2828, 1037, 1998, 11113, 5974, 6176, 102]),
       list([101, 2339, 2515, 2002, 2031, 18453, 2000, 2360, 2822, 7865, 2030, 18577, 19857, 4783, 3540, 8557, 2822, 2137, 2024, 102]),
       list([101, 1999, 2613, 2088, 5443, 11274, 2088, 2613, 2088, 2637, 2038, 2053, 2343, 5292, 3270, 18577, 19857, 8814, 4819, 102])],
      dtype=object)

In [ ]:
len(tokenized)

3733

In [ ]:
attention_mask

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [ ]:
# Through Bert

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)
    
features = last_hidden_states[0][:,0,:].numpy()
#labels = dataset["Label"]

In [ ]:
features

array([[-0.15585607,  0.5265683 , -0.06358972, ..., -0.21254304,
         0.43621236,  0.37098852],
       [-0.09106139, -0.06965308, -0.08741443, ..., -0.13359126,
         0.42896354,  0.41610092],
       [-0.17921373,  0.16137438, -0.27497485, ..., -0.3737376 ,
         0.5534895 ,  0.20148773],
       ...,
       [-0.12465265,  0.1322558 ,  0.16998237, ..., -0.6916145 ,
         0.64922667,  0.10799572],
       [-0.11429942,  0.19516681, -0.0261239 , ..., -0.39593658,
         0.62026715,  0.3506601 ],
       [-0.7328676 , -0.20874363,  0.14465337, ..., -0.57219785,
         0.60289717,  0.5307563 ]], dtype=float32)

In [ ]:
from numpy import savetxt
savetxt('features.csv', features, delimiter=',')

In [ ]:
from numpy import loadtxt
# load array
features = loadtxt('sample_data/features.csv', delimiter=',')
print(features)

[[-0.15585607  0.52656829 -0.06358972 ... -0.21254304  0.43621236
   0.37098852]
 [-0.09106139 -0.06965308 -0.08741443 ... -0.13359126  0.42896354
   0.41610092]
 [-0.17921373  0.16137438 -0.27497485 ... -0.3737376   0.55348951
   0.20148773]
 ...
 [-0.12465265  0.13225579  0.16998237 ... -0.69161451  0.64922667
   0.10799572]
 [-0.11429942  0.19516681 -0.0261239  ... -0.39593658  0.62026715
   0.35066009]
 [-0.7328676  -0.20874363  0.14465337 ... -0.57219785  0.60289717
   0.53075629]]


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
labels = dataset["Label"]
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size= 0.2) 

#Model Training

##Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr_clf.score(test_features, test_labels)

0.8982597054886211

In [ ]:
scores = cross_val_score(lr_clf, features, labels, cv=5)
mean, std = np.mean(scores), np.std(scores)
print("Mean: ", mean)
print("st: ", std)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


Mean:  0.8872282696469524
st:  0.01951381528173351


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# save the model to disk
import pickle
filename = 'lr_model.pkl'
pickle.dump(lr_clf, open(filename, 'wb'))

##Support Vector Machine

In [ ]:
from sklearn import svm

svm_clf = svm.SVC()
svm_clf.fit(train_features, train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
svm_clf.score(test_features, test_labels)

0.9236947791164659

In [ ]:
scores = cross_val_score(svm_clf, features, labels, cv=5)
scores

array([0.90227577, 0.89558233, 0.89022758, 0.92225201, 0.88739946])

In [ ]:
mean, std = np.mean(scores), np.std(scores)
print("Mean: ", mean)
print("st: ", std)

Mean:  0.899547430113663
st:  0.012434601642842286


In [ ]:
filename = 'svm_model.pkl'
pickle.dump(svm_clf, open(filename, 'wb'))

##Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(max_depth=2, random_state=0)
rf_clf.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
rf_clf.score(test_features, test_labels)

0.8058902275769746

In [ ]:
scores = cross_val_score(rf_clf, features, labels, cv=5)
mean, std = np.mean(scores), np.std(scores)
print("Mean: ", mean)
print("st: ", std)

Mean:  0.7980178084994132
st:  0.00046685116386778035


In [ ]:
filename = 'rf_model.pkl'
pickle.dump(rf_clf, open(filename, 'wb'))

##LSTM

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D, LSTM
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, Activation, Flatten
from keras.models import Model
from keras.initializers import Constant
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
MAX_SEQUENCE_LENGTH = 500
MAX_NUM_WORDS = 10000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2


x_train,x_test,y_train,y_test = train_test_split(dataset.Comment,dataset.Label,random_state = 42, test_size=VALIDATION_SPLIT, shuffle=True)

In [ ]:
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)

tokenizer.fit_on_texts(x_train)

# Transforms each text in texts to a sequence of integers. 
# So it basically takes each word in the text and replaces it with its corresponding integer value from the word_index dictionary.
# sequences = tokenizer.texts_to_sequences(news_data_df.text)
tokenized_train = tokenizer.texts_to_sequences(x_train)
X_train = pad_sequences(tokenized_train, maxlen=MAX_SEQUENCE_LENGTH)

word_index = tokenizer.word_index
print('Found {} unique tokens. and {} lines '.format(len(word_index), len(X_train)))

Found 15995 unique tokens. and 2986 lines 


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-02-01 16:46:09--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.186.253
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.186.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  66.0MB/s    in 25s     

2021-02-01 16:46:33 (63.7 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
from gensim.models.keyedvectors import KeyedVectors

tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = pad_sequences(tokenized_test, maxlen=MAX_SEQUENCE_LENGTH)

def get_embeddings(path):
  # model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300', binary=True, limit=500000)
  wv_from_bin = KeyedVectors.load_word2vec_format(path, binary=True, limit=500000) 
  #extracting word vectors from google news vector
  embeddings_index = {}
  for word, vector in zip(wv_from_bin.vocab, wv_from_bin.vectors):
      coefs = np.asarray(vector, dtype='float32')
      embeddings_index[word] = coefs

  return embeddings_index

embeddings_index = {}
embeddings_index = get_embeddings('/root/input/GoogleNews-vectors-negative300.bin.gz')
print('Found %s word vectors.' % len(embeddings_index))

Found 500000 word vectors.


In [ ]:
vocab_size = len(tokenizer.word_index) + 1

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
    try:
        embedding_vector = embeddings_index[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

In [ ]:
def lstm_net1():
    model = Sequential()

    #Non-trainable embeddidng layer
    model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
    
    model.add(LSTM(units=128 , return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=64))
    model.add(Dropout(0.1))
    model.add(Dense(units = 32 , activation = 'relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
#training an LSTM network
model2 = lstm_net1()

batch_size = 256
epochs = 8

model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 300)          4798800   
_________________________________________________________________
lstm (LSTM)                  (None, 500, 128)          219648    
_________________________________________________________________
dropout (Dropout)            (None, 500, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 3

In [ ]:
history = model2.fit(X_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs)

Epoch 1/8
12/12 [==============================] - 11s 213ms/step - loss: 0.5917 - accuracy: 0.7476 - val_loss: 0.4575 - val_accuracy: 0.8112
Epoch 2/8
12/12 [==============================] - 2s 155ms/step - loss: 0.4487 - accuracy: 0.8071 - val_loss: 0.3398 - val_accuracy: 0.8862
Epoch 3/8
12/12 [==============================] - 2s 156ms/step - loss: 0.3202 - accuracy: 0.8771 - val_loss: 0.2316 - val_accuracy: 0.9143
Epoch 4/8
12/12 [==============================] - 2s 156ms/step - loss: 0.2286 - accuracy: 0.9235 - val_loss: 0.2009 - val_accuracy: 0.9304
Epoch 5/8
12/12 [==============================] - 2s 157ms/step - loss: 0.1997 - accuracy: 0.9374 - val_loss: 0.1662 - val_accuracy: 0.9384
Epoch 6/8
12/12 [==============================] - 2s 156ms/step - loss: 0.1880 - accuracy: 0.9374 - val_loss: 0.1823 - val_accuracy: 0.9357
Epoch 7/8
12/12 [==============================] - 2s 156ms/step - loss: 0.1845 - accuracy: 0.9455 - val_loss: 0.1771 - val_accuracy: 0.9250
Epoch 8/8
12

In [ ]:
scores = model2.evaluate(X_test, y_test, verbose=0)
print(scores)

[0.14585021138191223, 0.9317269325256348]


In [ ]:
pred = model2.predict_classes(X_test)

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

F1_score = f1_score(y_test, pred, average='binary')
print("F1 score is", F1_score)

accuracy_score = accuracy_score(y_test, pred)
print("Acuracy is ", accuracy_score)

pred_proba = model2.predict(X_test)
auc_score = roc_auc_score(y_test, pred_proba)
print("AUC score is ", auc_score)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


F1 score is 0.8411214953271028
Acuracy is  0.9317269076305221
AUC score is  0.9816542288557213


##CNN

In [ ]:
def cnn_net1():
  model = Sequential()

  #Non-trainable embeddidng layer
  model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
  
  model.add(Dropout(0.2))
  model.add(Conv1D(filters=128, kernel_size=4, activation='relu'))
  model.add(GlobalMaxPooling1D())
  model.add(Dropout(0.2))
  model.add(Dense(units = 250 , activation = 'relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  
  return model


def get_pred_output(text_to_check):
  sequences = tokenizer.texts_to_sequences([text_to_check])
  data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
  predicted_val = model.predict_classes(data)
#     predicted_val = model.predict(data)    
#     if predicted_val.max() > 0.7:
#         output = 1
#     else:
#         output = 0
  return predicted_val

In [ ]:
# train a 1D convnet with global maxpooling
model_1 = cnn_net1()

batch_size = 256
epochs = 8

model_1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 500, 300)          4798800   
_________________________________________________________________
dropout_12 (Dropout)         (None, 500, 300)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 497, 128)          153728    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 128)               0         
_________________________________________________________________
dropout_13 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 250)               32250     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                

In [ ]:
history = model_1.fit(X_train, y_train, batch_size = batch_size , validation_data = (X_test,y_test) , epochs = epochs)

Epoch 1/8
12/12 [==============================] - 2s 101ms/step - loss: 0.5887 - accuracy: 0.7136 - val_loss: 0.4589 - val_accuracy: 0.8474
Epoch 2/8
12/12 [==============================] - 1s 86ms/step - loss: 0.4124 - accuracy: 0.8489 - val_loss: 0.2633 - val_accuracy: 0.9197
Epoch 3/8
12/12 [==============================] - 1s 87ms/step - loss: 0.2410 - accuracy: 0.9238 - val_loss: 0.1655 - val_accuracy: 0.9424
Epoch 4/8
12/12 [==============================] - 1s 87ms/step - loss: 0.1550 - accuracy: 0.9470 - val_loss: 0.1366 - val_accuracy: 0.9518
Epoch 5/8
12/12 [==============================] - 1s 87ms/step - loss: 0.1222 - accuracy: 0.9577 - val_loss: 0.1231 - val_accuracy: 0.9558
Epoch 6/8
12/12 [==============================] - 1s 86ms/step - loss: 0.1090 - accuracy: 0.9579 - val_loss: 0.1193 - val_accuracy: 0.9585
Epoch 7/8
12/12 [==============================] - 1s 86ms/step - loss: 0.0941 - accuracy: 0.9678 - val_loss: 0.1129 - val_accuracy: 0.9625
Epoch 8/8
12/12 [==

In [ ]:
scores = model_1.evaluate(X_test, y_test, verbose=0)
print(scores)

[0.1288660168647766, 0.9504685401916504]


In [ ]:
model_1.predict_classes(X_test)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
    

In [ ]:
pred_cnn = model_1.predict(X_test)
print(pred_cnn)

[[1.77312903e-02]
 [9.92492855e-01]
 [6.18563127e-03]
 [1.13052605e-02]
 [2.99340874e-01]
 [2.84968652e-02]
 [1.53999943e-02]
 [9.98461127e-01]
 [1.50174636e-03]
 [3.51566933e-02]
 [9.96542275e-01]
 [9.90034401e-01]
 [8.76157824e-03]
 [9.28854644e-01]
 [9.58765209e-01]
 [1.46801651e-01]
 [9.78841364e-01]
 [9.13175881e-01]
 [3.06353755e-02]
 [3.07155624e-02]
 [2.21062172e-03]
 [8.41315649e-03]
 [7.90550038e-02]
 [1.13877859e-02]
 [9.98171091e-01]
 [5.88451512e-03]
 [5.91182820e-02]
 [9.95435655e-01]
 [2.05185940e-03]
 [2.25114301e-02]
 [3.64532858e-01]
 [4.02421877e-03]
 [1.32809742e-03]
 [1.03266174e-02]
 [1.39385611e-02]
 [4.03946638e-03]
 [9.98969197e-01]
 [1.02338847e-02]
 [7.52434891e-04]
 [8.27380747e-04]
 [1.77344959e-02]
 [2.28713523e-03]
 [9.97455657e-01]
 [2.75022723e-03]
 [9.92912471e-01]
 [1.24521749e-02]
 [4.99738939e-02]
 [6.97553763e-03]
 [7.56829619e-01]
 [3.96183059e-02]
 [1.43483011e-02]
 [1.68975163e-02]
 [9.80452240e-01]
 [9.93416667e-01]
 [9.83962603e-03]
 [4.059610

In [ ]:
pred_cnn[2][0]

0.0061856313

In [ ]:
model_1.save('sample_data/cnn_model.h5')

In [ ]:
model_1.save('sample_data/cnn_model_95.h5')

In [ ]:
import tensorflow as tf

cnn_model = tf.keras.models.load_model('sample_data/cnn_model_95class.h5')

In [20]:
scores = cnn_model.evaluate(X_test, y_test, verbose=0)
print(scores)

[0.1208440363407135, 0.9544845819473267]


In [21]:
pred = cnn_model.predict_classes(X_test)

from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

F1_score = f1_score(y_test, pred, average='binary')
print("F1 score is", F1_score)

accuracy_score = accuracy_score(y_test, pred)
print("Acuracy is ", accuracy_score)

pred_proba = cnn_model.predict(X_test)
auc_score = roc_auc_score(y_test, pred_proba)
print("AUC score is ", auc_score)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


F1 score is 0.8843537414965987
Acuracy is  0.9544846050870147
AUC score is  0.9876428044960383
